In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pulses import RadarPulseGenerator, CommPulseGenerator
import torch
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)
t = torch.linspace(0, 1e-4, int(1e5), device=device)
dt = t[1] - t[0]
# Define parameters
T = 1e-4       # Total duration of radar pulse (1e-4 seconds)
B = 1e6        # Bandwidth of radar pulse (1 MHz)
K = 5         # Number of users (OFDMA users)
L = 100        # Number of bits in radar pulse duration
Tb = T / L     # Duration of each bit interval
Q = 256 #Number of Pulses in one Pulse train
T_PRI = T
carrier_freq = 800e6

myCommPulses = CommPulseGenerator(t, Tb, T, device)
myRadar = RadarPulseGenerator(t, T=T, device=device)

radar_pulse = myRadar.LFM_pulse(B) * np.sqrt(1e4) #normalize power to unity (1Watt)
radar_pulse_power = myRadar.power_calc(radar_pulse)
print(10*np.log10(radar_pulse_power.cpu().numpy()))

N0 = 10**(-120/10)
MN = int(200)
distance = 200
velocities = torch.logspace(1, 3, 10)
est_velocities = torch.zeros((MN, len(velocities)))
crlb_velocities = torch.zeros(len(velocities))
for idx, velocity in enumerate(velocities):
    print(f'------>Progress:{(idx+1)/len(velocities)*100}\r\nTesting Velocity={velocity}')
    est_velocities[:, idx] = myRadar.montecarlo_doppler_estimation_slow_time(radar_pulse, N0, distance, velocity, carrier_freq, 3e8, Q, T_PRI, MN)
    print(f'Mean value->{torch.mean(est_velocities[:, idx])}, \t Variance->{torch.var(est_velocities[:, idx])}')

# --- Modified Plotting Section ---
fig, ax1 = plt.subplots(figsize=(12, 7))

# Plotting on the primary (left) y-axis
ax1.set_title('Estimation Variance vs. Velocities', fontsize=16)
ax1.set_xlabel(r'Velocities ($m/s$)', fontsize=12)
ax1.set_ylabel(r'Variance ($\(m/s\)^2$)', fontsize=12)
ax1.grid(which='both', linestyle=':')
ax1.plot(velocities, torch.var(est_velocities, dim=0))

fig.tight_layout()
plt.savefig("crlb_vs_distance_with_snr.pdf", bbox_inches='tight')

cuda
4.3488213e-05
------>Progress:10.0
Testing Velocity=10.0


100%|██████████| 200/200 [03:09<00:00,  1.05it/s]


Mean value->8.89732551574707, 	 Variance->0.30021968483924866
------>Progress:20.0
Testing Velocity=16.681005477905273


100%|██████████| 200/200 [03:18<00:00,  1.01it/s]


Mean value->16.853906631469727, 	 Variance->0.13027171790599823
------>Progress:30.0
Testing Velocity=27.825593948364258


100%|██████████| 200/200 [03:18<00:00,  1.01it/s]


Mean value->27.55168914794922, 	 Variance->0.1963483989238739
------>Progress:40.0
Testing Velocity=46.415889739990234


 65%|██████▌   | 130/200 [02:09<01:14,  1.07s/it]